# Percobaan Rima

In [10]:
'''
The main optimization problem is split into three subproblems: 
1. optimizing the transmit beamforming, 
2. optimizing the phase shifts of RIS1 using SDR, 
2. optimizing the phase shifts of RIS2 using SOCP. 
The algorithm alternates between these three steps until convergence.


Bentuk akhir program
prev_power = calculate_power(w_c, w_r)
for iter in range(max_iter):
    # Langkah 2: Optimasi Beamforming
    # Langkah 3: Optimasi RIS1
    # Langkah 4: Optimasi RIS2
    # Langkah 5: Cek Konvergensi
    ...
'''

'\nThe main optimization problem is split into three subproblems: \n1. optimizing the transmit beamforming, \n2. optimizing the phase shifts of RIS1 using SDR, \n2. optimizing the phase shifts of RIS2 using SOCP. \nThe algorithm alternates between these three steps until convergence.\n\n\nBentuk akhir program\nprev_power = calculate_power(w_c, w_r)\nfor iter in range(max_iter):\n    # Langkah 2: Optimasi Beamforming\n    # Langkah 3: Optimasi RIS1\n    # Langkah 4: Optimasi RIS2\n    # Langkah 5: Cek Konvergensi\n    ...\n'

In [1]:
#Library
import numpy as np
import cvxpy as cp

In [2]:
# Parameter sistem (Simulasi paper)
M = 8  # Jumlah antena BS
K = 3  # Jumlah pengguna
N1 = 30  # Elemen RIS1
N2 = 30  # Elemen RIS2
T_R = 10  # Threshold SNR radar (dB)
Gamma_k = 2  # Minimum rate pengguna (bps/Hz)
epsilon = 1e-4   # Toleransi konvergensi

#ngarang
max_iter = 50  # Iterasi maksimum

## Langkah 1

In [3]:
# 1: Initialize the values of [w_c, w_r, v1,v2]
# nyoba random aja

'''beamforming'''
w_c = [np.random.randn(M, 1) + 1j*np.random.randn(M, 1) for _ in range(K)]
w_r = [np.random.randn(M, 1) + 1j*np.random.randn(M, 1) for _ in range(M)]
'''phase shift'''
v1 = np.exp(1j * 2 * np.pi * np.random.rand(N1))  # Unit modulus
v2 = np.exp(1j * 2 * np.pi * np.random.rand(N2))  # Unit modulus

## Langkah 2

In [4]:
#Definisi Variael CVXPY
W_c = [cp.Variable((M, 1), complex=True) for _ in range(K)]
W_r = [cp.Variable((M, 1), complex=True) for _ in range(M)]

In [5]:
#persamaan (2) Total transmit power
objective = sum(cp.norm(w, 2)**2 for w in W_c) + sum(cp.norm(w, 2)**2 for w in W_r)

In [6]:
# persamaan 3
# Parameter noise (Bagian IV)
sigma_R_dBm = -80       # Noise radar (dBm)
sigma_k_dBm = -80       # Noise pengguna (dBm)

# Konversi ke linear (Watt)
'''Konversi dBm ke Watt diperlukan karena CVXPY bekerja dalam domain linear (bukan dB).
P_Watt = 10^(PdBm/10)x10^-3
'''
sigma_R_sq = 10**(sigma_R_dBm / 10) / 1000  
sigma_k_sq = 10**(sigma_k_dBm / 10) / 1000  

# Constraint SNR radar (Persamaan 15)
# define A dulu dong sesuai persamaan 12
alpha = 1.0  # Sesuai nilai |α|^2 di makalah (contoh: 1)
# Definisikan saluran H_R sesuai model (misal: matriks random)
# Inisialisasi saluran H_R (contoh: matriks random)
H_R = np.random.randn(M, M) + 1j * np.random.randn(M, M)
A = alpha * H_R.conj().T @ H_R  # H_R^H * H_R

constraints = []
lhs = T_R * sigma_R_sq
rhs = sum(cp.real(cp.trace(A @ (W_c[k] @ W_c[k].H))) for k in range(K)) + \
      sum(cp.real(cp.trace(A @ (W_r[m] @ W_r[m].H))) for m in range(M))
constraints.append(rhs >= lhs)

In [7]:
# Definisikan problem
problem = cp.Problem(cp.Minimize(sum(cp.norm(w, 2)**2 for w in W_c) + sum(cp.norm(w, 2)**2 for w in W_r)), constraints)
problem.solve(solver=cp.ECOS)

DCPError: Problem does not follow DCP rules. Specifically:
The following constraints are not DCP:
1.0000000000000002e-10 <= real(trace([[ 1.04170233e+01+0.j          5.30916921e-02-3.01907949j ...
   1.19062018e-01+3.04674765j -8.79220414e+00-2.42532483j]
 [ 5.30916921e-02+3.01907949j  1.46595235e+01+0.j         ...
   1.05420935e-02+9.28075543j  4.09279051e+00-3.58915546j]
 ...
 [ 1.19062018e-01-3.04674765j  1.05420935e-02-9.28075543j ...
   2.54129531e+01+0.j         -2.18495096e+00-8.22758139j]
 [-8.79220414e+00+2.42532483j  4.09279051e+00+3.58915546j ...
  -2.18495096e+00+8.22758139j  2.60600371e+01+0.j        ]] @ var1 @ conj(var1).T)) + real(trace([[ 1.04170233e+01+0.j          5.30916921e-02-3.01907949j ...
   1.19062018e-01+3.04674765j -8.79220414e+00-2.42532483j]
 [ 5.30916921e-02+3.01907949j  1.46595235e+01+0.j         ...
   1.05420935e-02+9.28075543j  4.09279051e+00-3.58915546j]
 ...
 [ 1.19062018e-01-3.04674765j  1.05420935e-02-9.28075543j ...
   2.54129531e+01+0.j        

#itu bisa sih tapi error dcp gjls kata deepseek gini tp belum coba

'''
### Pastikan A PSD dengan menambahkan regularisasi kecil jika perlu
A = alpha * (H_R.conj().T @ H_R + 1e-6 * np.eye(M))

from cvxpy import quad_form

### Definisikan constraint radar SNR dengan quad_form
rhs = sum(quad_form(W_c[k], A) for k in range(K)) + \
      sum(quad_form(W_r[m], A) for m in range(M))
constraints.append(rhs >= lhs)


import numpy as np
import cvxpy as cp
from cvxpy import quad_form

### Parameter sistem
M = 8
K = 3
N1 = 30
N2 = 30
T_R = 10                  # Radar SNR threshold (dB)
Gamma_k = 2               # Minimum rate (bps/Hz)
epsilon = 1e-4
max_iter = 50

### Parameter noise
sigma_R_dBm = -80         
sigma_k_dBm = -80         
sigma_R_sq = 10**(sigma_R_dBm / 10) / 1000  
sigma_k_sq = 10**(sigma_k_dBm / 10) / 1000  

### Inisialisasi saluran H_R dan alpha
H_R = np.random.randn(M, M) + 1j * np.random.randn(M, M)
alpha = 1.0
A = alpha * (H_R.conj().T @ H_R + 1e-6 * np.eye(M))  # Pastikan A PSD

### Inisialisasi variabel beamforming
W_c = [cp.Variable((M, 1), complex=True) for _ in range(K)]
W_r = [cp.Variable((M, 1), complex=True) for _ in range(M)]

### Constraint radar SNR
constraints = []
lhs = T_R * sigma_R_sq
rhs = sum(quad_form(W_c[k], A) for k in range(K)) + \
      sum(quad_form(W_r[m], A) for m in range(M))
constraints.append(rhs >= lhs)

### Fungsi tujuan: Minimalkan total daya
objective = sum(cp.norm(w, 2)**2 for w in W_c) + sum(cp.norm(w, 2)**2 for w in W_r)
problem = cp.Problem(cp.Minimize(objective), constraints)

### Solve dengan solver SCS
problem.solve(solver=cp.SCS, verbose=True)

print("Optimasi berhasil! Status:", problem.status)

'''

## Referensi Deepseek

In [ ]:
import numpy as np
import cvxpy as cp

# Parameter sistem (contoh)
M = 8  # Jumlah antena BS
K = 3  # Jumlah pengguna
N1 = 30  # Elemen RIS1
N2 = 30  # Elemen RIS2
T_R = 10  # Threshold SNR radar (dB)
Gamma_k = 2  # Minimum rate pengguna (bps/Hz)
epsilon = 1e-4  # Toleransi konvergensi
max_iter = 50  # Iterasi maksimum

# Inisialisasi variabel (contoh acak)
w_c = [np.random.randn(M, 1) + 1j*np.random.randn(M, 1) for _ in range(K)]
w_r = [np.random.randn(M, 1) + 1j*np.random.randn(M, 1) for _ in range(M)]
v1 = np.exp(1j * 2 * np.pi * np.random.rand(N1))  # Unit modulus
v2 = np.exp(1j * 2 * np.pi * np.random.rand(N2))  # Unit modulus

# Simulasi saluran (diabaikan, perlu implementasi sesuai model makalah)
H_R = np.random.randn(M, M) + 1j*np.random.randn(M, M)
H_k = [np.random.randn(M, 1) + 1j*np.random.randn(M, 1) for _ in range(K)]

# Fungsi untuk menghitung daya pancar
def calculate_power(w_c, w_r):
    return sum(np.linalg.norm(w, 2)**2 for w in w_c) + sum(np.linalg.norm(w, 2)**2 for w in w_r)

# Algoritma utama
prev_power = calculate_power(w_c, w_r)
for iter in range(max_iter):
    # Langkah 5: Optimasi beamforming (SOCP)
    # Implementasi persamaan (17) menggunakan CVXPY
    W_c = [cp.Variable((M, 1), complex=True) for _ in range(K)]
    W_r = [cp.Variable((M, 1), complex=True) for _ in range(M)]
    
    constraints = []
    # Tambahkan constraints dari persamaan (15) dan (16)
    # ... (Implementasi rinci diperlukan sesuai model matematika)
    
    problem = cp.Problem(cp.Minimize(sum(cp.norm(w, 2)**2 for w in W_c) + sum(cp.norm(w, 2)**2 for w in W_r)),
                     constraints)
    problem.solve(solver=cp.ECOS)
    
    # Perbarui nilai beamforming
    w_c = [W_c[k].value for k in range(K)]
    w_r = [W_r[m].value for m in range(M)]
    
    # Langkah 6-8: Optimasi RIS1 (SDR dan Gaussian randomization)
    # Implementasi persamaan (32)
    U1 = cp.Variable((N1+1, N1+1), hermitian=True)
    constraints = [
        U1 >> 0,
        # Tambahkan constraints dari persamaan (27), (30), (31)
    ]
    problem = cp.Problem(cp.Minimize(0), constraints)
    problem.solve(solver=cp.SCS)
    
    # Gaussian randomization untuk ekstraksi v1
    # ... (Implementasi prosedur randomisasi)
    
    # Langkah 9: Optimasi RIS2 (SOCP)
    # Implementasi persamaan (38)
    u2 = cp.Variable(N2+1, complex=True)
    constraints = [
        # Tambahkan constraints dari persamaan (36) dan (37)
    ]
    problem = cp.Problem(cp.Minimize(0), constraints)
    problem.solve(solver=cp.ECOS)
    
    # Perbarui v2 dari u2
    v2 = u2.value[:N2] / np.abs(u2.value[:N2])  # Proyeksi ke unit modulus
    
    # Periksa konvergensi
    current_power = calculate_power(w_c, w_r)
    if abs(current_power - prev_power) < epsilon:
        break
    prev_power = current_power

print(f"Konvergensi tercapai dalam {iter+1} iterasi. Daya akhir: {current_power:.4f}")